<a href="https://colab.research.google.com/github/unnatii21/Data-Cleaning-Pipeline/blob/main/Data_Cleaning_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd

df = pd.read_csv('/content/loanprediction.zip')

df.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [5]:
# shape of the dataset
print(f"Rows: {df.shape[0]}, Columns: {df.shape[1]}")

Rows: 614, Columns: 13


In [6]:
# summary of the dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Loan_ID            614 non-null    object 
 1   Gender             601 non-null    object 
 2   Married            611 non-null    object 
 3   Dependents         599 non-null    object 
 4   Education          614 non-null    object 
 5   Self_Employed      582 non-null    object 
 6   ApplicantIncome    614 non-null    int64  
 7   CoapplicantIncome  614 non-null    float64
 8   LoanAmount         592 non-null    float64
 9   Loan_Amount_Term   600 non-null    float64
 10  Credit_History     564 non-null    float64
 11  Property_Area      614 non-null    object 
 12  Loan_Status        614 non-null    object 
dtypes: float64(4), int64(1), object(8)
memory usage: 62.5+ KB


In [7]:
# quick stats for numeric and object columns
print(df.describe(include='all'))

         Loan_ID Gender Married Dependents Education Self_Employed  \
count        614    601     611        599       614           582   
unique       614      2       2          4         2             2   
top     LP002990   Male     Yes          0  Graduate            No   
freq           1    489     398        345       480           500   
mean         NaN    NaN     NaN        NaN       NaN           NaN   
std          NaN    NaN     NaN        NaN       NaN           NaN   
min          NaN    NaN     NaN        NaN       NaN           NaN   
25%          NaN    NaN     NaN        NaN       NaN           NaN   
50%          NaN    NaN     NaN        NaN       NaN           NaN   
75%          NaN    NaN     NaN        NaN       NaN           NaN   
max          NaN    NaN     NaN        NaN       NaN           NaN   

        ApplicantIncome  CoapplicantIncome  LoanAmount  Loan_Amount_Term  \
count        614.000000         614.000000  592.000000         600.00000   
unique 

In [8]:
# count of missing values
missing = df.isnull().sum()
missing = missing[missing > 0].sort_values(ascending=False)
print(missing)

Credit_History      50
Self_Employed       32
LoanAmount          22
Dependents          15
Loan_Amount_Term    14
Gender              13
Married              3
dtype: int64


In [26]:
import pandas as pd

def handle_missing_values(df):
    # fill categorical columns with mode
    cat_cols = df.select_dtypes(include='object').columns
    for col in cat_cols:
        if df[col].isnull().sum() > 0:
            df.loc[:, col] = df[col].fillna(df[col].mode()[0])

    # fill numerical columns with median
    num_cols = df.select_dtypes(include=['int64', 'float64']).columns
    for col in num_cols:
        if df[col].isnull().sum() > 0:
            df.loc[:, col] = df[col].fillna(df[col].median())
    return df

# Apply the function to your DataFrame
df = handle_missing_values(df)


In [20]:
# example: convert 'LoanAmount' to float
df['LoanAmount'] = df['LoanAmount'].astype(float)

# convert categorical variables to category type
for col in cat_cols:
    df[col] = df[col].astype('category')

In [21]:
# strip whitespace and convert to consistent case
for col in cat_cols:
    df[col] = df[col].str.strip().str.lower()

In [22]:
# capping outliers at 99th percentile
for col in ['LoanAmount', 'ApplicantIncome']:
    upper = df[col].quantile(0.99)
    df[col] = df[col].apply(lambda x: upper if x > upper else x)

In [23]:
df.to_csv('cleaned_loan_data.csv', index=False)

In [31]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

def data_cleaning_pipeline(df):
    # 1. Handle Missing Values
    for col in df.columns:
        if df[col].dtype == 'object':  # Categorical columns
            if df[col].isnull().sum() > 0:
                mode_val = df[col].mode()[0]
                df.loc[:, col] = df[col].fillna(mode_val)
        else:  # Numerical columns
            if df[col].isnull().sum() > 0:
                median_val = df[col].median()
                df.loc[:, col] = df[col].fillna(median_val)

    # 2. Handle Outliers
    # Exclude boolean and object type columns from outlier handling
    numerical_cols_outliers = df.select_dtypes(include=['number']).columns #Selecting only numerical columns
    for col in numerical_cols_outliers:
        q1 = df[col].quantile(0.25)
        q3 = df[col].quantile(0.75)
        iqr = q3 - q1
        lower_bound = q1 - 1.5 * iqr
        upper_bound = q3 + 1.5 * iqr
        df = df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]

    # 3. Scaling
    scaler = StandardScaler()
    numerical_cols = df.select_dtypes(include=['int64', 'float64']).columns
    df[numerical_cols] = scaler.fit_transform(df[numerical_cols])

    return df

# Load your dataset
#df = pd.read_csv("your_data.csv")  # Replace with your actual file path or data loading
# Create a sample DataFrame (replace with your actual data loading)
data = {'A': [1, 2, 3, None, 5], 'B': [6, None, 8, 9, 10], 'C': ['a', 'b', 'a', None, 'c'], 'D': [True, False, True, False, True]}
df = pd.DataFrame(data)

df = pd.read_csv('/content/loanprediction.zip')
df_clean = data_cleaning_pipeline(df)
print(df_clean)

      Loan_ID  Gender Married Dependents     Education Self_Employed  \
0    LP001002    Male      No          0      Graduate            No   
1    LP001003    Male     Yes          1      Graduate            No   
2    LP001005    Male     Yes          0      Graduate           Yes   
3    LP001006    Male     Yes          0  Not Graduate            No   
4    LP001008    Male      No          0      Graduate            No   
..        ...     ...     ...        ...           ...           ...   
606  LP002961    Male     Yes          1      Graduate            No   
607  LP002964    Male     Yes          2  Not Graduate            No   
608  LP002974    Male     Yes          0      Graduate            No   
609  LP002978  Female      No          0      Graduate            No   
612  LP002984    Male     Yes          2      Graduate            No   

     ApplicantIncome  CoapplicantIncome  LoanAmount  Loan_Amount_Term  \
0           1.033966          -0.969937    0.077480           